In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv

In [2]:
# Initial imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import model_from_json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [3]:
## Load pickle for exports and imports of data  
import pickle 
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [4]:
## Import symbol list 
path = Path('../Resources/list_TI_export_success.pkl')
key_list = load_obj(path)
len(key_list)

932

In [5]:
test_symbol = key_list[10]
symbol = 'CPHI'
symbol

'CPHI'

In [6]:
## Load Data
# df = pd.read_csv(
#     '../FilesExportIndividualStockDFs_Big/'+symbol+'_combined_df.csv')
# df
path = Path('../FilesExport_DFs_with_TI_pkl/'+symbol+'_data_dict_with_technicals.pkl')
data_import = load_obj(path)

df = data_import[symbol].copy()

## Set X and y data 
X = df.drop(columns={'close','adjClose'}).values
y = df['close'].values

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

# load json and create model
file_path = Path('../Model_Data_low_acc/'+symbol+'_model_data.json')
with open(file_path, "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = Path('../Model_Data_low_acc/'+symbol+'_model_weights.h5')
loaded_model.load_weights(file_path)

## Compile loaded model and print mse score 
loaded_model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])
score = loaded_model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))
print()

## take dataframe copy and compare predicted values to actual values 
df2 = df.copy()
df2['predicted'] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]
df_pred

mse: 0.03%



,close,predicted
Date,,
2016-02-02,0.1200,0.117677
2016-02-03,0.1400,0.141602
2016-02-04,0.1400,0.140657
2016-02-05,0.1400,0.137877
2016-02-09,0.1300,0.131779
...,...,...
2021-10-25,0.5800,0.592874
2021-10-26,0.6376,0.638806
2021-10-27,0.6450,0.606741


In [7]:
score

[0.0002539549604989588, 0.0002539549604989588]

In [8]:
loaded_model.summary()

Model: "sequential_2329"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6986 (Dense)          (None, 8)                 168       
                                                                 
 dense_6987 (Dense)          (None, 8)                 72        
                                                                 
 dense_6988 (Dense)          (None, 1)                 9         
                                                                 
Total params: 249
Trainable params: 249
Non-trainable params: 0
_________________________________________________________________


In [10]:
df2 = df.copy()
#df2 = df2.tail(len(X_test_scaled))
df2['predicted'] = loaded_model.predict(X)
df_pred = df2[['close','predicted']]
df_pred

,close,predicted
Date,,
2016-02-02,0.1200,0.117677
2016-02-03,0.1400,0.141602
2016-02-04,0.1400,0.140657
2016-02-05,0.1400,0.137877
2016-02-09,0.1300,0.131779
...,...,...
2021-10-25,0.5800,0.592874
2021-10-26,0.6376,0.638806
2021-10-27,0.6450,0.606741


In [12]:
model_loss, model_accuracy = loaded_model.evaluate(X, y, verbose=0)
print(model_accuracy)

0.0002539549604989588
